In [ ]:
# default_exp core

# Data Preparation

> Prepare your data
> Apply your tranformations

### Import

In [ ]:
import sagemaker
import boto3
import pandas as pd
import io

In [ ]:
LOCAL_DIR = 'data'

### Input Raw Data S3 location

In [ ]:
s3_bucket = sagemaker.Session().default_bucket()
s3_input_prefix = 'heidelberg/data'
s3_input_filename = 'data.csv'

### Get dataframe from s3

In [ ]:
s3 = boto3.resource('s3')
obj = s3.Object(s3_bucket, s3_input_prefix+'/'+s3_input_filename)
body = obj.get()['Body'].read()
df=pd.read_csv(io.BytesIO(body))

### transfrom column name if(invalid)

In [ ]:
# input : old column name (string)
# output : transformed column name (string)a

def transform_column_name(column_name):
    allowed_values = "01234567890abcdefghijklmnopqrstuvwxyz"
    new_column_name = ""
    column_name=column_name.lower()
    count =0 
    for i in column_name:
        if i not in allowed_values:
            if count==0:
                continue
            new_column_name = new_column_name + '_'
            i='_'
        else :
            new_column_name = new_column_name + i
        count=count+1
        new_column_name = new_column_name[0:90]
        
    print ("new_column_name : ",new_column_name)
    return new_column_name

### Inserting Unique Identifier column as id

In [ ]:
l=len(df)
df['id']= [  float(i) for i in range(1,l+1)]

### Inserting timestamp

In [ ]:
import datetime as dt
import time
import dateutil.parser as parser

def generate_timestamps(n: int) -> list:
    start = dt.datetime.strptime('2020-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
    end = dt.datetime.strptime('2020-06-01 00:01:00', '%Y-%m-%d %H:%M:%S')
    timestamps = list()
    for _ in range(n):
        timestamp = faker.date_time_between(start_date=start, end_date=end, tzinfo=None).strftime('%Y-%m-%d %H:%M:%S')
        timestamps.append(timestamp)
    timestamps = sorted(timestamps)
    return timestamps

df['event_time'] = [float(time.time())]*l

### Move target column to start ( if you are using Xgboost)

In [ ]:
df = df[ ['target'] + [ col for col in df.columns if col != 'target' ] ]

### Drop unwanted columns

In [ ]:
del df['is_outlier']
list(df)

['target',
 'kst_brutto',
 'sm',
 'tm',
 'cl',
 'so3',
 'k2o',
 'na2o',
 'south_kiln_feed_01om886__tph__avg',
 'south_kiln_feed_01om886__tph__max',
 'north_kiln_feed_01om885__tph__avg',
 'north_kiln_feed_01om885__tph__max',
 'north_fan_speed_01oa943__rpm__avg',
 'north_fan_speed_01oa943__rpm__max',
 'south_fan_speed_02oa943__rpm__avg',
 'south_fan_speed_02oa943__rpm__max',
 'lignite_main_burner_03sk820__tph__avg',
 'lignite_main_burner_03sk820__tph__max',
 'bpg_main_burner_03bf810__tph__avg',
 'bpg_main_burner_03bf810__tph__max',
 'lignite_calciner_02sk820__tph__avg',
 'lignite_calciner_02sk820__tph__max',
 'bpg_calciner_02bf810__tph__avg',
 'bpg_calciner_02bf810__tph__max',
 'kbs_calciner_00kb950__tph__avg',
 'kbs_calciner_00kb950__tph__max',
 'total_energy_to_main_burner__gj_h__avg_0',
 'total_energy_to_main_burner__gj_h__max_0',
 'total_energy_to_main_burner__gj_h__avg_1',
 'total_energy_to_main_burner__gj_h__max_1',
 'total_energy_to_main_burner__gj_h__avg_2',
 'total_energy_to_mai

In [ ]:
df.to_csv(f'{LOCAL_DIR}/test.csv', index=False)

### Upload transfromed data to S3 bucket

In [ ]:
!aws s3 cp {LOCAL_DIR}/test.csv s3://{s3_bucket}/{s3_input_prefix}/transformed.csv

upload: data/test.csv to s3://sagemaker-us-east-1-082830052325/heidelberg/data/transformed.csv


In [ ]:
!rm {LOCAL_DIR}/test.csv

### (Optional) for Testing purpose

In [ ]:
def check():
    transformed_path = 'heidelberg/data/transfomed.csv'
    s3 = boto3.resource('s3')
    obj = s3.Object(s3_bucket, transformed_path)
    body = obj.get()['Body'].read()
    df=pd.read_csv(io.BytesIO(body))
    df.drop(df.filter(regex="Unname"),axis=1, inplace=True)